# MONAI Deploy App SDKによる可視化を含むセグメンテーションアプリの作成

MONAIで学習させたPyTorchモデルの臓器セグメンテーションアプリを作成し、Clara Vizとの連携でセグメンテーションと入力画像を可視化するチュートリアルです。

AIモデルのデプロイには、たとえ研究用であっても、臨床画像ネットワークとの連携が必要です。つまり、AIを導入するアプリケーションは、標準ベースの画像プロトコル、特に放射線画像についてはDICOMプロトコルをサポートする必要があるのです。

通常、DICOMネットワーク通信は、DICOM TCP/IPネットワークプロトコルまたはDICOMWebのいずれかで、DICOMデバイスまたはサービス（例：MONAI Deploy Informatics Gateway）によって処理されるので、展開アプリケーション自体は、入力としてDICOM Part10ファイルを使用して、AI結果をDICOM Part10ファイル（複数）に保存するだけで良いのです。セグメンテーションのユースケースでは、DICOMインスタンスファイルはDICOM SegmentationオブジェクトまたはDICOM RT Structure Setであり、分類ではDICOM Structure Reportおよび/またはDICOM Encapsulated PDFである可能性があります。

モデル学習中、入力画像とラベル画像は、通常、非DICOMボリューム画像形式、例えば、NIfTIとPNGで、特定のDICOMスタディシリーズから変換される。さらに、ボクセルスペーシングは、すべての画像で均一になるように再サンプリングされている可能性が最も高い。画像ネットワークと統合され、モダリティやPACSからDICOMインスタンスを受信する場合、AI導入アプリケーションは、複数のシリーズを持つDICOM検査全体を処理しなければならないことがあり、その画像の間隔は、学習したモデルによって予想と同じではないかもしれません。MONAI Deploy アプリケーション SDK は、このようなケースに一貫して効率的に対応するため、DICOM 検査を解析し、アプリケーション定義のルールに従って特定のシリーズを選択し、選択した DICOM シリーズをドメイン固有の画像フォーマットと関連する DICOM 属性を表すメタデータに変換するオペレータと呼ばれるクラスを提供します。

以下では、MONAI Deploy App SDK を使用した MONAI Deploy アプリケーションパッケージの作成方法について説明します。

:::{note}
ローカルテストでは、DICOM Part 10ファイルが不足している場合、3D Slicerなどのオープンソースプログラムを使用して、NIfTIをDICOMファイルに変換することが可能である。

[Clara Viz](https://github.com/NVIDIA/clara-viz): NVIDIA Clara Vizは、2D/3D医療画像データの視覚化のためのプラットフォームです。CUDAベースのレイトレーシングを用いた強力なボリューメトリック可視化を活用したアプリケーションを構築することができます。また、デジタル病理学で使用されるマルチ解像度画像の表示も可能です。

DICOM: DICOM（Digital Imaging and Communications in Medicine ダイコム）は、CTやMRI、CRの医用画像フォーマット、医用画像機器間で用いる通信プロトコルについて定義する、医用画像の共通規格である。 医用画像共通フォーマット、通信フォーマットであるDICOMによって、異なるメーカー間における医用画像機器の共有化を実現している。
:::

## ApplicationクラスでOperatorを作成し、接続する。

5つのOperatorで構成されるアプリケーションを実装します。

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: フォルダパス ([`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html))
    - **Output(dicom_study_list)**: メモリ上の DICOM スタディのリスト (List[[`DICOMStudy`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DICOMStudy.html)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: メモリ上の DICOM スタディのリスト (List[[`DICOMStudy`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DICOMStudy.html)])
    - **Input(selection_rules)**:  選択したルール (Dict)
    - **Output(study_selected_series_list)**:  メモリ上の DICOM series オブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**:  メモリ上の DICOM series オブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
    - **Output(image)**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
- **SpleenSegOperator**:
    - **Input(image)**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Output(seg_image)**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
- **DICOMSegmentationWriterOperator**:
    - **Input(seg_image)**: メモリ上のセグメンテーション画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Input(study_selected_series_list)**:  メモリ上の DICOM series オブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
    - **Output(dicom_seg_instance)**:  ファイルパス ([`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html))
- **ClaraVizOperator**:
    - **Input(image)**: メモリ上のボリューム画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Input(seg_image)**: メモリ上のセグメンテーション画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))


:::{note}
`DICOMSegmentationWriterOperator`は、患者のデモグラフィックや DICOM Study レベルの属性を使用するために、セグメンテーション画像とオリジナルの DICOM シリーズのメタデータの両方が必要です。

:::

アプリケーションのワークフローは次のようになる。

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB

    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : study_selected_series_list...study_selected_series_list
    DICOMSeriesToVolumeOperator --|> SpleenSegOperator : image...image
    DICOMSeriesSelectorOperator --|> DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    SpleenSegOperator --|> DICOMSegmentationWriterOperator : seg_image...seg_image
    DICOMSeriesToVolumeOperator --|> ClaraVizOperator : image...image
    SpleenSegOperator --|> ClaraVizOperator : seg_image...seg_image


    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        study_selected_series_list(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>study_selected_series_list : IN_MEMORY
        image(out) IN_MEMORY
    }
    class SpleenSegOperator {
        <in>image : IN_MEMORY
        seg_image(out) IN_MEMORY
    }
    class DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
    class ClaraVizOperator {
        <in>image : IN_MEMORY
        <in>seg_image : IN_MEMORY
    }
```

### 環境のセットアップ


In [ ]:
# MONAIなど、アプリケーションに必要な画像処理パッケージのインストール
!python -c "import monai" || pip install --upgrade -q "monai"
!python -c "import torch" || pip install -q "torch>=1.10.2"
!python -c "import numpy" || pip install -q "numpy>=1.21"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"
!python -c "import typeguard" || pip install -q "typeguard>=2.12.1"

# MONAI Deploy App SDK パッケージのインストール
!python -c "import monai.deploy" || pip install --upgrade -q "monai-deploy-app-sdk"

# Clara Viz パッケージのインストール
!python -c "import clara.viz" || pip install --upgrade -q "clara-viz"

Note: 更新されたパッケージを使用するために、Jupyterカーネルを再起動する必要がある場合があります。

### Google Driveからai_spleen_seg_dataをダウンロード／解凍する。

In [ ]:
# ai_spleen_bundle_dataテストデータのzipファイルをダウンロードする。
!pip install gdown
!gdown "https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ"

# Webブラウザまたはgdownでai_spleen_bundle_dataのzipファイルをダウンロードした後、ai_spleen_bundle_dataのzipファイルをダウンロードしてください。
!unzip -o "ai_spleen_seg_bundle_data.zip"

Downloading...
From: https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ
To: /home/mqin/src/monai-deploy-app-sdk/notebooks/tutorials/ai_spleen_seg_bundle_data.zip
100%|███████████████████████████████████████| 79.4M/79.4M [00:00<00:00, 102MB/s]
Archive:  ai_spleen_seg_bundle_data.zip
  inflating: dcm/1-001.dcm           
  inflating: dcm/1-002.dcm           
  inflating: dcm/1-003.dcm           
  inflating: dcm/1-004.dcm           
  inflating: dcm/1-005.dcm           
  inflating: dcm/1-006.dcm           
  inflating: dcm/1-007.dcm           
  inflating: dcm/1-008.dcm           
  inflating: dcm/1-009.dcm           
  inflating: dcm/1-010.dcm           
  inflating: dcm/1-011.dcm           
  inflating: dcm/1-012.dcm           
  inflating: dcm/1-013.dcm           
  inflating: dcm/1-014.dcm           
  inflating: dcm/1-015.dcm           
  inflating: dcm/1-016.dcm           
  inflating: dcm/1-017.dcm           
  inflating: dcm/1-018.dcm           
  inflating: dcm/1-

### importのセットアップ

アプリケーションとオペレータを定義するために必要なクラスやデコレータをインポートしましょう。

In [ ]:
import logging
from os import path

from numpy import uint8

import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    Invertd,
    LoadImaged,
    Orientationd,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
)

# SegmentDescription属性の設定に必要です。App SDK パッケージに含まれないため、直接インポートする。
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.clara_viz_operator import ClaraVizOperator

### モデル固有の推論Operatorクラスの作成

 [Operator](/modules/_autosummary/monai.deploy.core.Operator)クラスはOperatorクラスを継承し、 [@input](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.input.html)/[@output](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.output.html)デコレータで入出力のプロパティを指定する。

ビジネスロジックは<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html#monai.deploy.core.Operator.compute">compute()</a>メソッドで実装することになります。

App SDKでは、Torch Scriptモデルでセグメンテーション予測を行うために、`MonaiSegInferenceOperator`クラスが用意されています。このクラスは一貫性を保つために、MONAI辞書ベースの変換を `Compose`オブジェクトとして、変換の前と後に使用します。モデル固有の推論オペレータは、モデルの学習と検証で使用されたものに基づいて、プリとポスト変換の `Compose`を作成するだけでよいのです。デプロイアプリケーションでは、`ignite`は必要なく、サポートされていないことに注意してください。

#### SpleenSegOperator

`SpleenSegOperator`は、前段の`DICOMSeriesToVolumeOperator`によってDICOM CTシリーズから変換されたインメモリ[Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html)オブジェクトを入力とし、インメモリSegmentation  [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html)オブジェクトを出力として取得します。

`pre_process` 関数は、プリトランスフォームの `Compose` オブジェクトを生成します。`LoadImage` では、MONAI の `ImageReader` から派生した専用の `InMemImageReader` を用いて、メモリ内のピクセルデータを変換し、メタデータと同様に `numpy` 配列を返します。また、DICOM の入力ピクセルの間隔は、モデルで想定したものと異なることが多いので、 `Spacingd` 変換を用いて、想定した間隔で画像を再サンプリングする必要があります。

`post_process` 関数は、変換後のオブジェクトである `Compose` を生成します。`SaveImageD` 変換クラスは、セグメンテーションマスクを NIfTI 画像ファイルとして保存するために利用されます。これは、メモリ内のマスク画像が DICOM Segmentation ライターに渡され、DICOM Segmentation インスタンスを生成するために利用されるので、オプションとなります。また、 `Invertd` を用いて、セグメンテーション画像の向きと間隔を入力と同じにする必要があります。

`MonaiSegInferenceOperator` オブジェクトを作成する際には、`ROI` サイズを指定し、変換用の `Compose` オブジェクトも指定します。さらに、データセットの画像キー名を適宜設定します。

モデルのロードと予測の実行は `MonaiSegInferenceOperator` と他のSDKクラスでカプセル化されています。推論が完了すると、`MonaiSegInferenceOperator`によってセグメンテーション[Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html)オブジェクトが生成され、出力に設定されます(<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>)。

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["monai>=0.8.1", "torch>=1.5", "numpy>=1.21", "nibabel"])
class SpleenSegOperator(Operator):
    """DICOM CTシリーズから変換した3D画像で、脾臓のセグメンテーションを行います。
    """

    def __init__(self):

        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # この演算子はメモリ内のImageオブジェクトを取得するため、専用のImageReaderが必要です。
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # 推論と出力の保存を内蔵演算子に委ねる。
        infer_operator = MonaiSegInferenceOperator(
            (
                96,
                96,
                96,
            ),
            pre_transforms,
            post_transforms,
        )

        # ディクショナリーベースの変換で使用するキーの設定は変更される場合があります。
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # ここで、I/O仕様と実行コンテキストを使用して、組み込みオペレータに仕事を処理させます。
        infer_operator.compute(op_input, op_output, context)

    def pre_process(self, img_reader) -> Compose:
        """モデルで予測する前に、入力の前処理をするための変換を構成する。"""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Orientationd(keys=my_key, axcodes="RAS"),
                Spacingd(keys=my_key, pixdim=[1.5, 1.5, 2.9], mode=["bilinear"]),
                ScaleIntensityRanged(keys=my_key, a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
                EnsureTyped(keys=my_key),
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """予測結果の後処理を行うための変換を行う。"""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                Invertd(
                    keys=pred_key,
                    transform=pre_transforms,
                    orig_keys=self._input_dataset_key,
                    nearest_interp=False,
                    to_tensor=True,
                ),
                AsDiscreted(keys=pred_key, argmax=True),
                SaveImaged(
                    keys=pred_key,
                    output_dir=out_dir,
                    output_postfix="seg",
                    output_dtype=uint8,
                ),
            ]
        )


### アプリケーションクラスの作成

アプリケーションクラスは次のようなものです。

[Application](/modules/_autosummary/monai.deploy.core.Application)クラスを継承した `App` クラスを定義します。

リソースやパッケージの依存関係は [@resource](/modules/_autosummary/monai.deploy.core.resource) と [@env](/modules/_autosummary/monai.deploy.core.env) というデコレーターで指定します。

ベースクラスのメソッドである `compose` はオーバーライドされる。DICOM 解析、シリーズ選択（現在のリリースで最初のシリーズを選択）、ピクセルデータからボリューム画像への変換、セグメンテーションインスタンスの生成に必要なオブジェクトと、モデル固有の `SpleenSegOperator` が生成される。実行パイプラインは、これらのオブジェクトを <a href="../../modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>. で接続して Directed Acyclic Graph として作成される。

In [ ]:
@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        """アプリケーションのインスタンスを作成します。"""

        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # このメソッドは、実行するベースクラスを呼び出す。単に呼び出すだけであれば、省略可能です。
        self._logger.debug(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.debug(f"End {self.run.__name__}")

    def compose(self):
        """アプリ固有の演算子を作成し、処理DAG内で連鎖させる。"""

        self._logger.debug(f"Begin {self.compose.__name__}")
        # SDK 組み込みの演算子だけでなく、カスタムの演算子も作成します。
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(rules=Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # MONAI変換をサポートするモデル別推論演算子。

        # モデル別セグメンテーション演算子を作成
        spleen_seg_op = SpleenSegOperator()

        # 各セグメントに必要なセグメント記述と実際のアルゴリズムおよび関連する臓器/組織を提供するDICOM Seg ライターを作成する。
        # segment_label, algorithm_name, algorithm_version は 64 文字に制限されています。
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        # ユーザーは https://bioportal.bioontology.org/ontologies/SNOMEDCT などの SNOMED CT コードを検索できます。

        _algorithm_name = "3D segmentation of the Spleen from a CT series"
        _algorithm_family = codes.DCM.ArtificialIntelligence
        _algorithm_version = "0.1.0"

        segment_descriptions = [
            SegmentDescription(
                segment_label="Lung",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Lung,
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
        ]

        dicom_seg_writer = DICOMSegmentationWriterOperator(segment_descriptions)

        # 処理パイプラインを作成し、ソースとデスティネーションの演算子を指定し
        # 前者の出力が後者の入力と名前と型の両方で一致することを確認します。
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})

        # dicom_seg_writerは2つの入力を必要とし、それぞれがソースオペレータから来ることに注意してください。
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

        viz_op = ClaraVizOperator()
        self.add_flow(series_to_vol_op, viz_op, {"image": "image"})
        self.add_flow(spleen_seg_op, viz_op, {"seg_image": "seg_image"})

        self._logger.debug(f"End {self.compose.__name__}")

# これはJSONによるサンプルシリーズ選択ルールで、単純にCTシリーズを選択するものです。
# もし治験が複数のCTシリーズを持つ場合、それら全てが選択される。
# 詳細はDICOMSeriesSelectorOperatorを参照してください。
# 文字列のリストについては、例えば "ImageType": ["PRIMARY", "ORIGINAL"] のように、
# すべての要素がDICOM seriesの複数値属性にすべて含まれる場合にマッチングします。
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)",
                "ImageType": ["PRIMARY", "ORIGINAL"]
            }
        }
    ]
}
"""


## ローカルでアプリを実行する

Jupyterノートブック上でアプリを実行します。CT AbdomenシリーズのDICOMファイルが `dcm` に、Torch Scriptのモデルが `model.ts` に存在する必要があることに注意してください。あなたの環境での実際のパスを使用してください。

In [ ]:
app = AISpleenSegApp()

app.run(input="dcm", output="output", model="model.ts")

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1084421, Operator ID: ad334e7d-979b-484b-b73d-70bb012cfe05)


[2022-10-18 21:33:47,082] [INFO] (root) - Finding series for Selection named: CT Series
[2022-10-18 21:33:47,084] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-10-18 21:33:47,085] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-10-18 21:33:47,087] [INFO] (root) - On attribute: 'StudyDescription' to match value: '(.*?)'
[2022-10-18 21:33:47,087] [INFO] (root) -     Series attribute StudyDescription value: CT ABDOMEN W IV CONTRAST
[2022-10-18 21:33:47,088] [INFO] (root) - Series attribute string value did not match. Try regEx.
[2022-10-18 21:33:47,088] [INFO] (root) - On attribute: 'Modality' to match value: '(?i)CT'
[2022-10-18 21:33:47,089] [INFO] (root) -     Series attribute Modality value: CT
[2022-10-18 21:33:47,089] [INFO] (root) - Series attribute string value did not match. Try regEx.
[2022-10-18 21:33:47,090] [INFO] (root) - On att

Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 1084421, Operator ID: 52bc0253-84e7-4d83-bf67-aea10d6df3ae)
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 1084421, Operator ID: 47a7415b-28f3-4a81-abf8-43fa2bd55389)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator SpleenSegOperator
Executing operator SpleenSegOperator (Process ID: 1084421, Operator ID: 2963c156-eb97-43ec-b48d-ddbc67984cc1)
Converted Image object metadata:
SeriesInstanceUID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239, type <class 'str'>
SeriesDate: 20090831, type <class 'str'>
SeriesTime: 101721.452, type <class 'str'>
Modality: CT, type <class 'str'>
Serie

/home/mqin/src/monai-deploy-app-sdk/.venv/lib/python3.8/site-packages/highdicom/valuerep.py:54: UserWarning: The string "C3N-00198" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  warnings.warn(
[2022-10-18 21:34:04,374] [INFO] (highdicom.seg.sop) - add plane #0 for segment #1
[2022-10-18 21:34:04,377] [INFO] (highdicom.seg.sop) - add plane #1 for segment #1
[2022-10-18 21:34:04,378] [INFO] (highdicom.seg.sop) - add plane #2 for segment #1
[2022-10-18 21:34:04,381] [INFO] (highdicom.seg.sop) - add plane #3 for segment #1
[2022-10-18 21:34:04,382] [INFO] (highd

Done performing execution of operator DICOMSegmentationWriterOperator

Going to initiate execution of operator ClaraVizOperator
Executing operator ClaraVizOperator (Process ID: 1084421, Operator ID: 22570e7b-4760-478a-b876-78cc803f90c3)


Box(children=(Widget(), VBox(children=(interactive(children=(Dropdown(description='View mode', index=2, option…

Done performing execution of operator ClaraVizOperator



Jupyter notebook内でアプリケーションが確認できたら、上記のPythonコードをアプリケーションフォルダ内のPythonファイルに書き込んでいきます。

アプリケーションフォルダは以下のような構成になります。

```bash
my_app
├── __main__.py
├── app.py
└── spleen_seg_operator.py
```

:::{note}
複数のファイルを作成するのではなく、ファイルの内容を含む一つのアプリケーションPythonファイル（`spleen_app.py`など）を作成すればよいのです。
このような例は、<a href="./02_mednist_app.html#executing-app-locally">MedNist Classifier Tutorial</a>で見ることができます。
:::

In [ ]:
# アプリケーションフォルダーの作成
!mkdir -p my_app

### spleen_seg_operator.py

In [ ]:
%%writefile my_app/spleen_seg_operator.py
import logging
from os import path

from numpy import uint8

import monai.deploy.core as md
from monai.deploy.core import ExecutionContext, Image, InputContext, IOType, Operator, OutputContext
from monai.deploy.operators.monai_seg_inference_operator import InMemImageReader, MonaiSegInferenceOperator
from monai.transforms import (
    Activationsd,
    AsDiscreted,
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    Invertd,
    LoadImaged,
    Orientationd,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
)


@md.input("image", Image, IOType.IN_MEMORY)
@md.output("seg_image", Image, IOType.IN_MEMORY)
@md.env(pip_packages=["monai>=0.8.1", "torch>=1.10.2", "numpy>=1.21", "nibabel"])
class SpleenSegOperator(Operator):
    """DICOM CTシリーズから変換した3D画像で、脾臓のセグメンテーションを行います。
    """

    def __init__(self):

        self.logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__()
        self._input_dataset_key = "image"
        self._pred_dataset_key = "pred"

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):

        input_image = op_input.get("image")
        if not input_image:
            raise ValueError("Input image is not found.")

        output_path = context.output.get().path

        # この演算子はメモリ内のImageオブジェクトを取得するため、専用のImageReaderが必要です。
        _reader = InMemImageReader(input_image)
        pre_transforms = self.pre_process(_reader)
        post_transforms = self.post_process(pre_transforms, path.join(output_path, "prediction_output"))

        # 推論と出力の保存を内蔵演算子に委ねる。
        infer_operator = MonaiSegInferenceOperator(
            (
                96,
                96,
                96,
            ),
            pre_transforms,
            post_transforms,
        )

        # ディクショナリーベースの変換で使用するキーの設定は変更される場合があります。
        infer_operator.input_dataset_key = self._input_dataset_key
        infer_operator.pred_dataset_key = self._pred_dataset_key

        # ここで、I/O仕様と実行コンテキストを使用して、組み込みオペレータに仕事を処理させます。
        infer_operator.compute(op_input, op_output, context)

    def pre_process(self, img_reader) -> Compose:
        """モデルで予測する前に、入力の前処理をするための変換を構成する。"""

        my_key = self._input_dataset_key
        return Compose(
            [
                LoadImaged(keys=my_key, reader=img_reader),
                EnsureChannelFirstd(keys=my_key),
                Orientationd(keys=my_key, axcodes="RAS"),
                Spacingd(keys=my_key, pixdim=[1.5, 1.5, 2.9], mode=["bilinear"]),
                ScaleIntensityRanged(keys=my_key, a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
                EnsureTyped(keys=my_key),
            ]
        )

    def post_process(self, pre_transforms: Compose, out_dir: str = "./prediction_output") -> Compose:
        """予測結果の後処理を行うための変換を行う。"""

        pred_key = self._pred_dataset_key
        return Compose(
            [
                Activationsd(keys=pred_key, softmax=True),
                Invertd(
                    keys=pred_key,
                    transform=pre_transforms,
                    orig_keys=self._input_dataset_key,
                    nearest_interp=False,
                    to_tensor=True,
                ),
                AsDiscreted(keys=pred_key, argmax=True),
                SaveImaged(
                    keys=pred_key,
                    output_dir=out_dir,
                    output_postfix="seg",
                    output_dtype=uint8,
                ),
            ]
        )


Overwriting my_app/spleen_seg_operator.py


### app.py

In [ ]:
%%writefile my_app/app.py
import logging

from spleen_seg_operator import SpleenSegOperator

# SegmentDescription属性の設定に必要です。App SDK パッケージに含まれないため、直接インポートする。
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.clara_viz_operator import ClaraVizOperator

# これはJSONによるサンプルシリーズ選択ルールで、単純にCTシリーズを選択するものです。
# もしスタディに複数のCTシリーズがある場合は、それらすべてが選択されます。
# 詳しくはDICOMSeriesSelectorOperatorをご覧ください。
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)",
                "ImageType": ["PRIMARY", "ORIGINAL"],
            }
        }
    ]
}
"""

@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        """アプリケーションのインスタンスを作成します。"""

        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # このメソッドは、実行するベースクラスを呼び出す。単に呼び出すだけであれば、省略可能です。
        self._logger.debug(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.debug(f"End {self.run.__name__}")

    def compose(self):
        """アプリ固有の演算子を作成し、処理DAG内で連鎖させる。"""

        self._logger.debug(f"Begin {self.compose.__name__}")
        # SDK 組み込みの演算子だけでなく、カスタムの演算子も作成します。
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(rules=Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # MONAI変換をサポートするモデル別推論演算子。

        # モデル別セグメンテーション演算子を作成
        spleen_seg_op = SpleenSegOperator()

        # DICOM Seg writer を作成し、各セグメントに必要なセグメント記述を実際のアルゴリズムと関連する臓器/組織とともに提供する。
        # segment_label, algorithm_name, algorithm_version は 64 文字に制限されています。 
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html 
        # ユーザーは https://bioportal.bioontology.org/ontologies/SNOMEDCT などの SNOMED CT コードを検索できます。

        _algorithm_name = "3D segmentation of the Spleen from a CT series"
        _algorithm_family = codes.DCM.ArtificialIntelligence
        _algorithm_version = "0.1.0"

        segment_descriptions = [
            SegmentDescription(
                segment_label="Lung",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Lung,
                algorithm_name=_algorithm_name,
                algorithm_family=_algorithm_family,
                algorithm_version=_algorithm_version,
            ),
        ]

        dicom_seg_writer = DICOMSegmentationWriterOperator(segment_descriptions)

        # 処理パイプラインを作成し、ソースとデスティネーションの演算子を指定し、
        # 前者の出力が後者の入力と名前と型の両方で一致することを確認します。
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})

        # dicom_seg_writerは2つの入力を必要とし、それぞれがソースオペレータから来ることに注意してください。
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

        viz_op = ClaraVizOperator()
        self.add_flow(series_to_vol_op, viz_op, {"image": "image"})
        self.add_flow(spleen_seg_op, viz_op, {"seg_image": "seg_image"})

        self._logger.debug(f"End {self.compose.__name__}")

# これはJSONのシリーズ選択ルールのサンプルで、単純にCTシリーズを選択するものです。
# 例えば "ImageType": ["PRIMARY", "ORIGINAL"] の場合、
# 全ての要素がDICOM seriesの複数値属性に含まれていれば一致することになります。
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "Modality": "(?i)CT",
                "ImageType": ["PRIMARY", "ORIGINAL"],
                "PhotometricInterpretation": "MONOCHROME2"
            }
        }
    ]
}
"""


if __name__ == "__main__":
    # アプリを作成し、スタンドアロンでテストします。このモードで実行する場合、以下の点に注意してください。
    #     -i <DICOM folder>, 入力DICOM CTシリーズフォルダ
    #     -o <output folder>, 出力フォルダ（デフォルトは$PWD/output)
    #     -m <model file>, モデルファイルのパス
    # 例.
    #     python3 app.py -i input -m model.ts
    #
    AISpleenSegApp(do_run=True)

Overwriting my_app/app.py


```python
if __name__ == "__main__":
    AISpleenSegApp(do_run=True)
```

上記の行は `python` インタープリタを使ってアプリケーションコードを実行するために必要なものです。

### \_\_main\_\_.py

\_\_main\_\_.py は、<a href="../../developing_with_sdk/packaging_app.html#required-arguments">MONAI Application Packager</a>が、アプリケーションフォルダのパス（例：`python simple_imaging_app`）でアプリケーションを実行したときに、メインアプリケーションコード（`app.py`）を検出するために必要なものです。

In [ ]:
%%writefile my_app/__main__.py
from app import AISpleenSegApp

if __name__ == "__main__":
    AISpleenSegApp(do_run=True)

Overwriting my_app/__main__.py


In [ ]:
!ls my_app

app.py	__main__.py  __pycache__  spleen_seg_operator.py


This time, let's execute the app in the command line.

In [ ]:
!python my_app -i dcm -o output -m model.ts

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1084688, Operator ID: 110251db-4c50-42ff-a56d-32bd876bb739)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 1084688, Operator ID: 741a6c5d-8439-414f-b4f0-c499fa9f85a9)
[2022-10-18 21:34:15,156] [INFO] (root) - Finding series for Selection named: CT Series
[2022-10-18 21:34:15,156] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-10-18 21:34:15,156] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-10-18 21:34:15,156] [INFO] (root) - On attribute: 'Modality' to match value: '(?i)CT'
[2022-10-18 21:34:15,156] [INFO] (root) -     Series attribute Modality value: CT
[2022-10-18 21:34:15,156

上記のコマンドは、以下のコマンドラインと同じです。

In [ ]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
!monai-deploy exec my_app -i dcm -o output -m model.ts

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 1084773, Operator ID: 21b70bc7-bd07-4803-936b-aadc983343c8)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 1084773, Operator ID: 06cf792c-e49a-4a84-b04b-c38ec1e2830a)
[2022-10-18 21:34:41,042] [INFO] (root) - Finding series for Selection named: CT Series
[2022-10-18 21:34:41,042] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-10-18 21:34:41,042] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-10-18 21:34:41,042] [INFO] (root) - On attribute: 'Modality' to match value: '(?i)CT'
[2022-10-18 21:34:41,042] [INFO] (root) -     Series attribute Modality value: CT
[2022-10-18 21:34:41,043

In [ ]:
!ls output

1.2.826.0.1.3680043.10.511.3.11636838214613793635775978376672891.dcm
1.2.826.0.1.3680043.10.511.3.17384468917290596349831996191635582.dcm
1.2.826.0.1.3680043.10.511.3.55545104192889656878608836519404425.dcm
prediction_output


## Packaging app

アプリケーションに追加されたClara-Vizオペレータは、インタラクティブな可視化に使用されるため、[MONAI Application Packager](/developing_with_sdk/packaging_app)でパッケージ化されていないこと。